In [ ]:
# Utilização da técnica SMOTE para oversampling das imagens

In [211]:
import numpy as np
import pandas as pd
import cv2
import random
import os
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

In [333]:
path1 = os.getcwd()+'//..//features//'
path2 = os.getcwd()+'//..//features2//'

In [422]:
data1 = pd.read_csv(path1+'all_features.csv', sep=';', index_col=0)
data2 = pd.read_csv(path2+'all_features.csv', sep=';', index_col=0)

sst = pd.read_csv(path1+'/../reference_values/primeiros_dados_palmer_sst.csv', sep=';', index_col=0)
firmeza = pd.read_csv(path1+'/../reference_values/primeiros_dados_palmer_firmeza.csv', sep=';', index_col=0)
time = pd.read_csv(path1+'/../reference_values/primeiros_dados_palmer_time.csv', sep=';', index_col=0)

In [423]:
data = pd.concat([(data1.loc[data1.index.values[600:]]), \
                   data2], axis=0, sort=True)

In [432]:
data.loc[:60,'sst'] = sst.iloc[600:]['sst'].values
data.loc[:60,'firmeza'] = firmeza.iloc[600:]['firmeza'].values
data.loc[:60,'time'] = time.iloc[600:]['time'].values

In [299]:
k = 5
N = 200

In [436]:
nums = np.arange(1501,1501+int(data.shape[0]*(N/100)))
indexes = [str(num)+'_synthetic' for num in nums]

In [461]:
df = smote(data, k, N, indexes, 1501, ['time', 'sst', 'firmeza'])

[235 227 220 226 222]
0_10_B_diff      17.165721
0_10_G_diff      12.411845
0_10_H_diff     -20.261520
0_10_L_diff       9.236761
0_10_R_diff       2.881520
0_10_S_diff    -139.969513
0_10_V_diff       0.046538
0_10_a_diff      -4.946979
0_10_b_diff     -12.826425
0_11_B_diff      17.982293
0_11_G_diff      12.130932
0_11_H_diff     -21.266350
0_11_L_diff       9.499074
0_11_R_diff       4.309771
0_11_S_diff    -140.311262
0_11_V_diff       1.388149
0_11_a_diff      -3.887220
0_11_b_diff     -13.063692
0_12_B_diff      18.609821
0_12_G_diff      12.624025
0_12_H_diff     -19.779214
0_12_L_diff      10.736466
0_12_R_diff       6.644150
0_12_S_diff    -134.952928
0_12_V_diff       3.896075
0_12_a_diff      -2.864040
0_12_b_diff     -12.491173
0_13_B_diff      19.819398
0_13_G_diff      13.866159
0_13_H_diff     -18.108332
                  ...     
region_7_L       36.507865
region_7_R       49.887367
region_7_S      146.070116
region_7_V       49.957643
region_7_a      134.129236
region

KeyError: 'time'

In [460]:
def smote(data, k, N, indexes, i, targets):
    N = (int)(N/100)
    n_samples = data.shape[0]
    features = data.drop(columns=targets)
    synthetic = pd.DataFrame(index=indexes,columns=data.columns.values)
    
    neigh = NearestNeighbors(n_neighbors=k+1)
    neigh.fit(features)
    
#     def treat_targets(neighbors, sample, targets):
#         sm = 0
#         calc_targ = []
        
#         for target in targets:
#             sm = 0
#             for neigh in range(neighbors):
#                 sm = sm + (sample[target]-features.iloc[neigh][target])
    
    def normalize_features():
        return features.apply(lambda x: x/np.max(abs(x)))
    
#     def post_normalize_features():
    def get_neighbors(sample):
        return neigh.kneighbors([sample], return_distance=False)[0][1:]
        
    def populate(norm, neighbors, sample, i):
        for __ in range(N):
            rnd = random.randint(0,k-1)
            
            diff = features.iloc[neighbors[rnd]] - features.iloc[sample]
            gap = random.uniform(0,1)
            feature_row = features.iloc[sample] + diff*gap
            
            synthetic_target = data.iloc[sample][targets]
            
            synthetic_sample = pd.Series(data=feature_row, index=data.columns.values, name=str(i)+'_synthetic')
            
            print(synthetic_sample)
            for target in targets:
                synthetic_sample[target] = feature_row[target]
            
            synthetic.loc[str(i)+'_synthetic'] = synthetic_sample
            
            i=i+1
        return i
        
    norm = normalize_features()
    
    for sample in range(n_samples):
        neighbors = get_neighbors(norm.iloc[sample])
        print(neighbors)
        i = populate(norm, neighbors, sample, i)
    
    return synthetic

In [452]:
synthetic_sample = pd.Series(data=0, index=['a', 'b'], name=str(i)+'_synthetic')